Just like the $k$-means algorithm for tabular data, the $k$-means algorithm for time series data aims to partition a set of time series into $k$ clusters, where each time series belongs to the cluster with the nearest mean (centroid) time series.
 
The algorithm works as follows:
 
1. **Initialization**: Randomly select $k$ initial centroids from the time series dataset.
 
1. **Assignment Step**: Assign each time series to the cluster whose centroid is closest, based on a chosen similarity measure (e.g., Euclidean distance, Dynamic Time Warping).
 
1. **Update Step**: Recalculate the centroids of the clusters by taking the mean of all time series assigned to each cluster.
 
1. **Repeat**: Repeat the assignment and update steps until convergence (i.e., when assignments no longer change or a maximum number of iterations is reached).
 
In practice, calculating the centroid of a cluster needs to be compatible with the chosen similarity measure (at least it is very beneficial if it is compatible).
Especially for non-Euclidean measures with time series of variable length like Dynamic Time Warping, specialized methods may be used to compute the centroid.

Here we will have a look at an example using the `sktime` library.
The dataset we will use is a subset of the `trace` dataset, which is a synthetic dataset introduced by @roverso2002 for the purpose of plant diagnostics.
The subset includes only 4 classes of univariate time series.

In [117]:
import numpy as np
import plotly.graph_objs as go

from sklearn.preprocessing import StandardScaler
from sktime.clustering.k_means import TimeSeriesKMeansTslearn
from tslearn.datasets import CachedDatasets

In [41]:
X_train, y_train, X_test, y_test = CachedDatasets().load_dataset("Trace")

# Combine train and test sets since clustering does not require a train-test split
X = np.concatenate((X_train, X_test))
y = np.concatenate((y_train, y_test))

In [14]:
X.shape

(200, 275, 1)

In [42]:
y.shape

(200,)

We have 200 time series in total, each of length 275.

Note that we also have class labels, which is not the case in real clustering problems.
We will solely use them in the end to evaluate the clustering performance.

In [23]:
fig = go.Figure()
for i in range(X.shape[0]):
    fig.add_trace(go.Scatter(y=X[i, :, 0], mode='lines', line=dict(width=1, color='grey'), opacity=0.2, showlegend=False))

fig.update_layout(title='All Time Series in X', xaxis_title='Time', yaxis_title='Value', height=400)
fig.show()

Without class labels it is hard to count the number of classes in the data, but we can see that there are some patterns in the data.

Since normalization and scaling is important for distance-based methods, we will use the `StandardScaler` from `sklearn` to standardize the data to have zero mean and unit variance.

In [ ]:
X_scaled = StandardScaler().fit_transform(X[:, :, 0])  # In this case, the data set was already scaled beforehand, but we do it here explicitely for demonstration purposes

Let us also just visualize a few time series from the dataset to get a better idea of the data.

In [103]:
COLORS = ['#A6CEE3', '#B2DF8A', '#FDBF6F', '#CAB2D6']  # pastel, colorblind-friendly

sampled_ids = [0, 10, 25, 81]

fig = go.Figure()

for idx, i in enumerate(sampled_ids):
    fig.add_trace(go.Scatter(
        y=X[i, :, 0],
        mode='lines',
        line=dict(width=3, color=COLORS[idx]),
        name=f"Sample {i}"
    ))

fig.update_layout(title='Sampled Time Series from X', xaxis_title='Time', yaxis_title='Value', height=400)
fig.show()

### Euclidean Distance Example

In this example we know that there are 4 classes in the data, so we will set $k=4$.

Luckily, `tslearn` already implements a variety of clustering algorithms that we can use out of the box, including the $k$-means algorithm.

In [133]:
k = 4  # number of clusters

clusterer = TimeSeriesKMeansTslearn(n_clusters=4, metric="euclidean", random_state=42)
y_predicted = clusterer.fit_predict(X)

/home/noxthot/projects/own_lectures/ws25_industrial_datascience_manuscript/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/noxthot/projects/own_lectures/ws25_industrial_datascience_manuscript/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



*Helper function for plotting clusters and cluster centers*

In [134]:
def plot_clusters(X, y, title):
    fig = go.Figure()

    for cluster_idx, cluster in enumerate(sorted(set(y))):
        idx = np.where(y == cluster)[0]
        show_legend = True  # Only show legend for the first trace of each cluster
        
        for i in idx:
            fig.add_trace(go.Scatter(
                y=X[i, :, 0],
                mode='lines',
                line=dict(width=1, color=COLORS[cluster_idx]),
                opacity=0.5,
                name=f'Cluster {cluster_idx + 1}' if show_legend else None,
                legendgroup=cluster_idx,
                showlegend=show_legend
            ))
            show_legend = False

    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Value',
        height=500
    )

    fig.show()


def plot_centroids(clusterer):
    fig = go.Figure()

    for cluster_idx, centroid in enumerate(clusterer.cluster_centers_):
        fig.add_trace(go.Scatter(
            y=centroid[:, 0],
            mode='lines',
            line=dict(width=3, color=COLORS[cluster_idx]),
            name=f'Centroid {cluster_idx + 1}'
        ))

    fig.update_layout(
        title='Cluster Centroids',
        xaxis_title='Time',
        yaxis_title='Value',
        height=400
    )
    
    fig.show()

Next, we plot the $k$-means clusters.
Remember that the legend is clickable.

In [135]:
plot_clusters(X, y_predicted, 'Time Series clustered by k-means (euclidean distance)')

In comparison, here are the true classes according to the labels in the dataset.

In [136]:
plot_clusters(X, y, 'Time Series with true labels')

The four classes from the original data set can be described as follows:
- Class A (Cluster 1 in Figure): Time series that start high, rise to a huge peak around the middle, fall back to low and then gradually rise to high again.
- Class B (Cluster 2 in Figure): Time series that start high, drop to a low point around the middle, and then rise back up.
- Class C (Cluster 3 in Figure): Time series that start low, quickly rise to high around the middle, and then show some oscillations on high plateau
- Class D (Cluster 4 in Figure): Similiar to Class C, but without oscillations.

The clusters found by the $k$-means algorithm do not correspond well to these classes:
- Class A and B are mixed up in Cluster 2.
- Class C and D are mixed up in Cluster 1, 3, and 4.

This is due to the fact that the $k$-means algorithm is based on minimizing the within-cluster distances based on the euclidean distance, which does not necessarily correspond to the true classes in the data.

This also reflects when we visualize the cluster centers.

In [137]:
plot_centroids(clusterer)

Next, let us check how the clustering performs with dynamic time warping as distance measure.

### Dynamic Time Warping

While euclidean distance also uses the mean for computing the centroid, dynamic time warping uses a more complex method called soft-DTW barycenter averaging, which is compatible with the DTW distance measure.
More information on soft-DTW can be found in the related paper @{cuturi2017}.

In [141]:
k = 4  # number of clusters

clusterer_dwt = TimeSeriesKMeansTslearn(n_clusters=4, metric="softdtw", n_jobs=-1, random_state=1337)  # n_jobs=-1 uses all available CPU cores
y_predicted_dwt = clusterer_dwt.fit_predict(X)

/home/noxthot/projects/own_lectures/ws25_industrial_datascience_manuscript/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/noxthot/projects/own_lectures/ws25_industrial_datascience_manuscript/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



Have a look at the execution times of both algorithms.
While DTW is executed in parallel (`n_jobs=-1`), it is still significantly slower than the euclidean distance version.
The reason for this is that the DTW algorithm got a time complexity of $O(NM)$, where $N$ and $M$ are the lengths of the two time series to be compared.
More sophisticated algorithms are able to slightly reduce this time complexity, but so far, the runtime complexity stays quadratic.

However, the clustering results look better now.

In [142]:
plot_clusters(X, y_predicted_dwt, 'Time Series clustered by k-means (dynamic time warping)')

In [143]:
plot_centroids(clusterer_dwt)

Cluster 2 matches class A very well, cluster 4 matches class B.
However, cluster 1 and 3 still contain a mix of class C and D.

## Elbow method for determining the number of clusters

One open question is how to determine the number of clusters $k$.
A common method is the elbow method, which plots the sum of distances to the nearest cluster center for different values of $k$.
The idea is to choose the value of $k$ at which the rate of decrease sharply shifts, forming an elbow shape in the plot.

In [153]:
sum_of_distances = []
K = range(2, 10)

for k in K:
    km = TimeSeriesKMeansTslearn(
                          n_clusters=k,
                          metric="euclidean",
                          random_state=1337,
                          n_jobs=-1,
    )
    
    km = km.fit(X)
    sum_of_distances.append(km.inertia_)

/home/noxthot/projects/own_lectures/ws25_industrial_datascience_manuscript/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/noxthot/projects/own_lectures/ws25_industrial_datascience_manuscript/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/noxthot/projects/own_lectures/ws25_industrial_datascience_manuscript/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/home/noxthot/projects/own_lectures/ws25_industrial_datascience_manuscript/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [154]:
import plotly.graph_objs as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(K),
    y=sum_of_distances,
    mode='lines+markers',
    marker=dict(color='blue'),
    line=dict(color='blue'),
    name='Sum of distances'
))

fig.update_layout(
    title='Elbow Method For Optimal k',
    xaxis_title='k',
    yaxis_title='Sum of distances'
)

fig.show()

The elbow method suggests that four clusters is a good choice for $k$, which matches the true number of classes in the data.

Often it makes sense to start with a larger number of clusters and then merge similar clusters later on.
This allows to capture more subtle patterns in the data.

## Additional readings

[tslearn dtw documentation (accessed: 29 09 2025)](https://tslearn.readthedocs.io/en/stable/user_guide/dtw.html) briefly introduces dynamic time warping, barycenters and soft-DTW.
To learn more about different methods used to calculate barycenters, have a look at the [tslearn barycenter documentation (accessed: 29 09 2025)](https://tslearn.readthedocs.io/en/latest/auto_examples/clustering/plot_barycenters.html#sphx-glr-auto-examples-clustering-plot-barycenters-py).

To learn more about dynamic time warping, the [wikipedia article on dynamic time warping (accessed: 29 09 2025)](https://en.wikipedia.org/wiki/Dynamic_time_warping) is a good starting point.

$k$-means clustering in time series is still a very active research area.
A recent preprint by @holder2024 gives a nice overview over the variants of $k$-means for time series and discusses their pros and cons.